## 문제 1

### 1-1. 분류 평가지표 중 '정밀도'와 '재현율'에 대한 개념과 이 둘은 어떤 관계인지 설명해주세요.

* 정밀도 : 예측을 positive로 한 대상 중에 예측과 실제 값이 positive로 일치한 데이터의 비율.
* 재현율 : 실제값이 positive인 대상 중 예측과 실제 값이 positive로 일치한 데이터 비율.
* 이 둘은 trade - off 관계이다. 즉 재현율이 올라가면 정밀도는 낮아지고, 재현율이 낮아지면 정밀도는 증가한다.

### 1-2.  '정밀도'와 '재현율'의 관계로 인하여 다른 어떤 평가지표를 쓰는 것이 좋은지 이유와 함께 설명해주세요.

```
F1 score
이유 : 정밀도와 재현율은 trade-off관계이므로 이 둘의 수치가 적절하게 조화가 되는 평가 지표가 필요하다. 
이때 이 둘의 조화평균으로 계산되는 F1 score는 정밀도와 재현율이 어느 한쪽으로 치우치지 않은 수치를 지닐 때 상대적으로 높은 값을 갖게 된다.
따라서 F1-score를 쓰는 것이 더 좋다.
```

***

## 문제 2

### 정밀도와 재현율이 각각 더 중요한 사례를 들어주세요.
- 정밀도가 중요한 사례와 재현율이 중요한 사례 총 2가지를 적어주셔야 합니다.
- 되도록이면 PPT 이외의 사례를 들어주세요!

정밀도
* 스팸 메일 판별

재현율
* 금융 사기 적발 모델 : 실제 금융거래 사기인 Positive를 negative로 잘못 판단하게 되면 회사에 미치는 손해가 클 것이다. 

***

## 문제3

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import Binarizer

In [67]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

### [ 종양종류 예측해보기 ]
- load_breast_cancer은 위스콘신 유방암 데이터로 악성종양(malignant)과 양성종양(benign)으로 구분되는 데이터세트 입니다.
- 타깃 레이블 값의 종류는 악성이 '0'으로 양성이 '1'로 되어 있습니다.

#### 기본 제공 코드

In [68]:
# 데이텃셋 로드 및 X,y 지정
dataset = load_breast_cancer()
X_features=dataset.data
y_label=dataset.target

In [69]:
# `train_test_split`으로 train, test 데이터 분리

X_train,X_test,y_train,y_test = train_test_split(X_features, y_label, test_size=0.3, random_state=11)
print(X_train.shape, X_test.shape)

(398, 30) (171, 30)


In [70]:
# RandomForest 객체 생성 후 예측 수행

from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state = 0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
pred_proba = rf_clf.predict_proba(X_test)[:,1]

### 3-1. 정확도, 정밀도, 재현율, 오차행렬을 산출하세요.
- get_clf_eval 함수를 활용하세요.

In [71]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}'.format(accuracy, precision, recall))
    
get_clf_eval(y_test, pred)

오차 행렬
[[ 56   5]
 [  1 109]]
정확도 : 0.9649, 정밀도 : 0.9561, 재현율 : 0.9909


### 3-2. 임계값이 0.3, 0.5, 0.7, 0.9일때 오차행렬, 정확도, 정밀도, 재현율을 구하세요.

In [72]:
thresholds = [0.3, 0.5, 0.7, 0.9]

In [73]:
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임계값 : ', custom_threshold)
        get_clf_eval(y_test, custom_predict)
        print('')
        
get_eval_by_threshold(y_test, pred_proba.reshape(-1, 1), thresholds)

임계값 :  0.3
오차 행렬
[[ 51  10]
 [  0 110]]
정확도 : 0.9415, 정밀도 : 0.9167, 재현율 : 1.0000

임계값 :  0.5
오차 행렬
[[ 56   5]
 [  1 109]]
정확도 : 0.9649, 정밀도 : 0.9561, 재현율 : 0.9909

임계값 :  0.7
오차 행렬
[[ 59   2]
 [  4 106]]
정확도 : 0.9649, 정밀도 : 0.9815, 재현율 : 0.9636

임계값 :  0.9
오차 행렬
[[59  2]
 [11 99]]
정확도 : 0.9240, 정밀도 : 0.9802, 재현율 : 0.9000



### 3-3. 위의 임곗값들 중 어떤 것이 가장 적절할까요? 답과 그 이유를 적어주세요.

0.7 : 정밀도와 재현율 값이 가장 비슷하면서 높기 때문.

***

## 문제 4

### 4-1. F1 스코어의 개념과 어떨 때 상대적으로 높은 값를 가지는지 적어주세요.

* F1 score : 정밀도와 재현율을 결합한 지표로 이 둘의 조화평균으로 구합니다.
* F1 score는 정밀도와 재현율이 어느 한 쪽으로 치우치지 않는 수치를 나타낼 때 상대적으로 높은 값을 가집니다.

### 4-2. F1 스코어를 구하세요.

In [74]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pred)
print('F1 스코어 : {0:.4f}'.format(f1))

F1 스코어 : 0.9732


### 4-3. `get_clf_eval()` 함수에 F1 스코어를 구하는 코드를 추가하세요.

In [75]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    
    # F1 스코어 추가
    f1 = f1_score(y_test, pred)
    
    print('오차 행렬')
    print(confusion)
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, F1 : {3:.4f}'.format(accuracy, precision, recall, f1))
    
get_clf_eval(y_test, pred)

오차 행렬
[[ 56   5]
 [  1 109]]
정확도 : 0.9649, 정밀도 : 0.9561, 재현율 : 0.9909, F1 : 0.9732


### 4-4. <U>F1 스코어 >= 0.98</U>이 되도록 값을 출력하세요.  (다른 평가 지표의 값은 무시)
  
`Hint : 임계값 조정`

In [76]:
def f1_98(y_test, pred_proba_c1, thresholds):
    
    for custom_threshold in thresholds:
        
        binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        
        f1 = f1_score(y_test, custom_predict)
        if f1 >= 0.98:
            print('임계값 : ', custom_threshold)
            print('F1 : {0:.4f}'.format(f1))
        
thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65]
f1_98(y_test, pred_proba.reshape(-1, 1), thresholds)

임계값 :  0.6
F1 : 0.9818


***

## 문제 5

### 5-1. ROC-AUC가 무엇인지 설명하세요. 

In [77]:
# ROC 곡선 : 거짓 양성 비율에 대한 진짜 양성 비율의 곡선
# TPR(True Positive Rate) : 실제 양성을 양성으로 예측하는 비율
# TNR(True Negative Rate) : 실제 음성을 음성으로 예측하는 비율
# AUC(Area Under Curve) : ROC 곡선의 면적 값

### 5-2. ROC-AUC 값을 구하세요. 

In [78]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, pred_proba)
print('AUC : {:.4f}'.format(roc_auc))

AUC : 0.9903
